## Char-RNN

In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

52   71707


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 100
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=30)

Epoch 1/30
2014/2014 [==============================] - 697s 341ms/step - loss: 2.2574
Epoch 2/30
2014/2014 [==============================] - 676s 335ms/step - loss: 1.5608
Epoch 3/30
2014/2014 [==============================] - 697s 346ms/step - loss: 1.4486
Epoch 4/30
2014/2014 [==============================] - 685s 339ms/step - loss: 1.3942
Epoch 5/30
2014/2014 [==============================] - 654s 324ms/step - loss: 1.3611
Epoch 6/30
2014/2014 [==============================] - 654s 324ms/step - loss: 1.3339
Epoch 7/30
2014/2014 [==============================] - 655s 325ms/step - loss: 1.3144
Epoch 8/30
2014/2014 [==============================] - 651s 323ms/step - loss: 1.2980
Epoch 9/30
2014/2014 [==============================] - 654s 324ms/step - loss: 1.2862
Epoch 10/30
2014/2014 [==============================] - 680s 337ms/step - loss: 1.2739
Epoch 11/30
2014/2014 [==============================] - 653s 324ms/step - loss: 1.2639
Epoch 12/30
2014/2014 [==================

In [14]:
model.save('char_rnn_lyrics')

In [15]:
tf.executing_eagerly()

True

In [16]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [17]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [18]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [19]:
print(complete_text("a", temperature=1))

aunt have but soil composed
to be
and the shing exchanged down around to be our own falts, and you mu


In [20]:
print(complete_text("b", temperature=1))

become broken embracion

fall we've always down abandon my body

we are drawn for decision
i am no ev


In [21]:
print(complete_text("f", temperature=1))

f shawed is remains
and sway rest to take troughts

and i know where to take my flaws

[chorus]
i bel


## Stateful RNN

In [22]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=n_steps, drop_remainder=True)

In [23]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [24]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [54]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])

In [55]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [56]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [57]:
history = model.fit(dataset, epochs=200, callbacks=[ResetStatesCallback()], steps_per_epoch=steps_per_epoch)

Epoch 1/200
20/20 [==============================] - 17s 666ms/step - loss: 3.7173
Epoch 2/200
20/20 [==============================] - 12s 620ms/step - loss: 3.0408
Epoch 3/200
20/20 [==============================] - 13s 653ms/step - loss: 2.9894
Epoch 4/200
20/20 [==============================] - 12s 620ms/step - loss: 2.9558
Epoch 5/200
20/20 [==============================] - 13s 635ms/step - loss: 2.8790
Epoch 6/200
20/20 [==============================] - 13s 661ms/step - loss: 2.7885
Epoch 7/200
20/20 [==============================] - 13s 647ms/step - loss: 2.6561
Epoch 8/200
20/20 [==============================] - 13s 660ms/step - loss: 2.5617
Epoch 9/200
20/20 [==============================] - 13s 648ms/step - loss: 2.4924
Epoch 10/200
20/20 [==============================] - 13s 646ms/step - loss: 2.4342
Epoch 11/200
20/20 [==============================] - 13s 653ms/step - loss: 2.3838
Epoch 12/200
20/20 [==============================] - 13s 643ms/step - loss: 2.3399
E

20/20 [==============================] - 14s 683ms/step - loss: 1.4938
Epoch 99/200
20/20 [==============================] - 14s 697ms/step - loss: 1.4880
Epoch 100/200
20/20 [==============================] - 14s 679ms/step - loss: 1.4925
Epoch 101/200
20/20 [==============================] - 13s 649ms/step - loss: 1.4894
Epoch 102/200
20/20 [==============================] - 14s 690ms/step - loss: 1.4838
Epoch 103/200
20/20 [==============================] - 14s 681ms/step - loss: 1.4815
Epoch 104/200
20/20 [==============================] - 14s 678ms/step - loss: 1.4776
Epoch 105/200
20/20 [==============================] - 14s 702ms/step - loss: 1.4762
Epoch 106/200
20/20 [==============================] - 14s 690ms/step - loss: 1.4771
Epoch 107/200
20/20 [==============================] - 14s 682ms/step - loss: 1.4699
Epoch 108/200
20/20 [==============================] - 14s 691ms/step - loss: 1.4651
Epoch 109/200
20/20 [==============================] - 14s 682ms/step - loss: 1.

20/20 [==============================] - 13s 654ms/step - loss: 1.3460
Epoch 195/200
20/20 [==============================] - 14s 681ms/step - loss: 1.3451
Epoch 196/200
20/20 [==============================] - 14s 674ms/step - loss: 1.3402
Epoch 197/200
20/20 [==============================] - 14s 710ms/step - loss: 1.3414
Epoch 198/200
20/20 [==============================] - 14s 690ms/step - loss: 1.3363
Epoch 199/200
20/20 [==============================] - 14s 689ms/step - loss: 1.3371
Epoch 200/200
20/20 [==============================] - 14s 717ms/step - loss: 1.3350


In [58]:
model.save('stateful_rnn_lyrics')

In [63]:
tf.executing_eagerly()

True

In [64]:
#stateless copy to allow us to use different batch sizes
stateless_model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [65]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [66]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [67]:
print(complete_text("t"))

to harn
i love me alone
i'll see you could fall see
no suppised on a may breeds of help me to this to


In [68]:
print(complete_text("m"))

more waught that never with beo eads
shay of that open broken hearf
i am trying to the bodding foothe
